# Image Similarity: AML Package for Computer Vision
A large number of problems in the computer vision domain can be solved by ranking images according to their similarity. For example, retail companies want to show customers products which are similar to the ones bought in the past. Or companies with large amounts of data want to organize and search their images effectively.

This notebook shows how the AML Package for Computer vision can be used to train, evaluate, and deploy an image similarity model. Example images and annotations are provided, but the reader can bring their own dataset and train their own unique ranker. Currently, CNTK is used as the deep learning framework. Training is peformed locally on a GPU powered machine ([Deep Learning VM](https://azuremarketplace.microsoft.com/en-us/marketplace/apps/microsoft-ads.dsvm-deep-learning?tab=Overview)), and deployment uses the Azure ML Operationalization CLI.

It is encouraged to first try the **Image Classification** tutorial before running this tutorial as many features from Image Classification are used.

The following steps are performed:
1. Dataset Creation
2. Image Pairs
3. Model Definition and Training
4. Evaluation and Visualzation
5. Webservice Deployment

## Overview
Our approach to measure image similarity is visualized at a high level as shown by the diagram below:

<img src="https://github.com/Azure/ImageSimilarityUsingCntk/blob/master/doc/pipeline.jpg?raw=true" width=800>

* Given two images, we want to measure the visual distance between them. For the purpose of this tutorial distance and similarity are used interchangebly (since similarity can be computed by simply taking the negative of the distance). 
* Optionally, in a pre-processing step, one can detect the object-of-interest and crop the image to that area. Please see the Object Detection notebook for how to train and evaluate such a model based on an approach called Faster-RCNN. 
* Each image is then represented using the output of a DNN which was pre-trained on millions of images. The input to the DNN is simply the image itself, and the output is the penultimate layer (512 floating point values for the ResNet18 model). 
* These 512-floats image representations are then scaled to each have an L2 norm of one, and the visual distance between two images is defined as a function of their respective 512-floats vectors. Possible distance metrics include the L1 or the L2 norm. The advantage of these metrics is that they are non-parametric and therefore do not require any training. The disadvantage however is that each of the 512 dimensions is assumed to carry the same amount of information which in practice is not true. Hence, we use a Linear SVM to train a weighted L2 distance, which can give a higher weight to informative dimensions, and vice versa down-weight dimensions with little or no information.
* Finally, the output of the system is the desired visual similarity for the given image pair.

Using the output of a DNN is powerful and is shown to give good results on a wide variety of tasks. However, better results can be achieved through fine-tuning the network before computing the image representations. This approach is known as [transfer learning](https://en.wikipedia.org/wiki/Transfer_learning): starting with an original model trained on a large dataset of generic images, it is then fine-tuned in this tutorial using a small set of images from our own dataset. 

In [ ]:
# Imports
import warnings
warnings.filterwarnings("ignore")
import os, json, shutil, cntk
import cvtk
from cvtk.utils import Constants
from cvtk.core import Context, ClassificationDataset, Image, Label, Splitter, CNTKTLModel
from cvtk.core.ranker import ImagePairs, ImageSimilarityMetricRanker, ImageSimilarityLearnerRanker, ImageSimilarityRandomRanker, RankerEvaluation
from cvtk.utils.ranker_utils import visualize_ranked_images
from cvtk.augmentation import augment_dataset
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

## Step 1: Dataset Creation

The recommended way to generate a Dataset object in CVTK is by providing the root directory of the images on the local disk. This directory has to follow the same general structure as the tableware dataset in CVTK's image classification notebook, ie. contain sub-directories with the actual images:
- root
    - label1
    - label2
    - ...
    - labeln
  
Using a different dataset is therefore as easy as changing the root path `dataset_location` in the code below to point at different images, and to set `dataset_name` to any user-chosen string.

#### Set the local storage context
Note AML has a limit of 25 MB, so we will set the CVTK outputs directory to be outside of the workbench.

In [ ]:
# Set storage context.
out_root_path = "../../../cvtk_output"
Context.create(outputs_path=out_root_path, persistent_path=out_root_path, temp_path=out_root_path)

# If the user wants to use a local context without a json file run the lines below
#if 'AZUREML_NATIVE_SHARE_DIRECTORY' not in os.environ:
#    os.environ['AZUREML_NATIVE_SHARE_DIRECTORY'] = './share'
#context = Context.get_global_context()

#### Download the Dataset and Split
We will download a small upper body clothing texture dataset of around 330 images, annotated in one of 3 different textures: dotted, striped, leopard. The figure below shows examples for the attributes of the dotted (left two columns), striped (middle two columns), and leopard (right two columns). It is important to note that the annotations were done according to the upper body clothing item. The ranker needs to learn to focus on the relevant part of the image and to ignore all other areas (e.g. pants, shoes). 

|<h3><center>Dotted</center></h3>|<h3><center>Striped</center></h3>|<h3><center>Leopard</center></h3>|
|:-------------:|:-------------:|:-----:|
| <img src="https://github.com/Azure/ImageSimilarityUsingCntk/blob/master/doc/examples_dotted.jpg?raw=true" width=250> | <img src="https://github.com/Azure/ImageSimilarityUsingCntk/blob/master/doc/examples_striped.jpg?raw=true" width=250> | <img src="https://github.com/Azure/ImageSimilarityUsingCntk/blob/master/doc/examples_leopard.jpg?raw=true" width=250> |

In [ ]:
# Set dataset name and location
dataset_name = "fashion"
dataset_location = os.path.join(Context.get_global_context().storage.outputs_path, "data", dataset_name)

# A dataset location can also be specfied as shown here
# dataset_location=r"../classification/sample_data/imgs_recycling" 
# dataset_name = "my_data"

In [ ]:
# This will download around 330 images. This cell only needs to be executed once.
import download_images
print("Downloading images to: " + dataset_location)
download_images.download_all(dataset_location)

All images are assigned for either training or for testing - this split is mutually exclusive. Here we use a ratio of 0.5, 50% of the images from each attribute are assigned to training, and 50% to testing. 

In [ ]:
dataset = ClassificationDataset.create_from_dir(dataset_name, dataset_location)
print("Dataset consists of {} images with {} labels.".format(len(dataset.images), len(dataset.labels)))
# Split the data into train and test
train_set, test_set = dataset.split(train_size = .5, random_state=1, stratify="label")
print("Number of original training images = {}.".format(train_set.size()))

## Step 2: Image Pairs Generation

Image pairs are used to train and evaluate the image ranker. We select up to `num_train_sets=60` query images from each of the 3 attributes. Each query image is paired with one image from the same attribute, and up to
`num_ref_images_per_set=50` images from other attributes. This leads to a maximum of 60\*50 = 3000 mostly negative image pairs. For testing the same approach is used. More details of the implementation can be seen in the documenation.

Shown below are randomly generated image pairs for a given query image in the top row: (left) positive pair since the clothing texture in both images is dotted; (middle and right) negative pairs where the images have different textures. 


|<h3><center>Positive</center></h3>|<h3><center>Negative</center></h3>|<h3><center>Negative</center></h3>|
|:-------------:|:-------------:|:-----:|
| <img src="https://github.com/Azure/ImageSimilarityUsingCntk/blob/master/doc/example_pair_pos.jpg?raw=true" width=150> | <img src="https://github.com/Azure/ImageSimilarityUsingCntk/blob/master/doc/example_pair_neg2.jpg?raw=true" width=150> | <img src="https://github.com/Azure/ImageSimilarityUsingCntk/blob/master/doc/example_pair_neg1.jpg?raw=true" width=150> |


In [ ]:
# If you get errors running this due to downloads try reducing num_different_label
num_train_sets = 60
num_test_sets = 60
num_ref_images_per_set = 50
train_pairs = ImagePairs(train_set, num_train_sets, num_ref_images_per_set)
print('There are {} sets of image pairs generated for all labels from training data.'.format(len(train_pairs.image_sets)))
test_pairs = ImagePairs(test_set, num_test_sets, num_ref_images_per_set)
print('There are {} sets of image pairs generated for all labels from training data.'.format(len(test_pairs.image_sets)))

## Step 3: Model Definition and Training: 
### Load pre-trained DNN model and optionally refine it
This code refines a pre-trained DNN using the training set from previous steps. Note that this can be slow even with a GPU. Hence, optionally, the pre-trained DNN can be used as-is which however will produce suboptimal image representations, and reduce ranker accuracy.

In [ ]:
refine_DNN = True # Use the pretrained model as-is or refine
model = CNTKTLModel(train_set.labels, class_map = {i: l.name for i, l in enumerate(dataset.labels)}, base_model_name='ResNet18_ImageNet_CNTK')
if refine_DNN:
    model.train(train_set)

### Image Similarity Ranker
Instantiate (and if required train) the image similarity ranker. These rankers internally compare two images using e.g. the L2 distance of the 512-floats image representations, or an SVM which was trained to score how similar two images are.

In [ ]:
similarity_method = "l2" # Options: "random", "L2", "svm"

if similarity_method == "random":
    ranker = ImageSimilarityRandomRanker()
elif similarity_method == "l2":
    ranker = ImageSimilarityMetricRanker(model, metric="l2")
elif similarity_method == "svm":
    from sklearn.svm import LinearSVC
    svm_learner = LinearSVC(C = 0.01) # SVM-defined weighted L2-distance. Need to train, but this is fast.
    ranker = ImageSimilarityLearnerRanker(model, learner=svm_learner)

# Train the ranker, random and L2 do not need training and .train() will do nothing
ranker.train(train_pairs)

### Evaluation 

Quantitative evaluation is performed using ImagePairs, where each query image is paired with 1 positive and 50 negative images. These 51 reference images are sorted using their distance to the query image. Then the rank of the positive image within the 51 images is computed. Rank 1 corresponds to the best possible result, rank 51 to the worst. Random guessing would on average produce a rank of 25. 

The diagram below, after sorting, shows where the postive image has rank of 3 (note that this example uses 100 negative images):
<img src="https://github.com/Azure/ImageSimilarityUsingCntk/blob/master/doc/example_ranking.jpg?raw=true" width=800> 

In [ ]:
re = RankerEvaluation(ranker, test_pairs)
acc_top_1 = re.compute_accuracy(top_n = 1)
acc_top_5 = re.compute_accuracy(top_n = 5)
mean_rank = re.compute_mean_rank()
median_rank = re.compute_median_rank()

print('Top 1 accuracy: {} %'.format(str(acc_top_1)))
print('Top 5 accuracy: {} %'.format(str(acc_top_5)))
print('Mean rank: {}'.format(str(mean_rank)))
print('Median rank: {}'.format(str(median_rank)))

In [ ]:
acc_plot = re.top_n_acc_plot(n=32, visualize=True)

### Visualize the Results
Given an image pair, we can compare its query image (left) to all reference images and show the reference image with lowest distance (middle), as well as the positive reference image in the image pair (right). In the best case scenario where the positive image has the lowest distance, both the middle and right will show the same image. 

An example is shown below:

In [ ]:
visualized_results_plots = re.visualize_results(n=2, visualize=True)

### Top Related Images Given a Query Image
Here we can visualize the related images given a query image. If using a deployed cluster, you can also pass the serialized output and get the related images using the same function.

In [ ]:
import cv2
ranker.set_reference_data(test_set) # need to set the reference set to test against
image_path = test_set.images[0].storage_path # take an example image to score against
input_image = cv2.imread(image_path) # requires an image read in an OpenCV format
similar_images_plots = re.visualize_similar_images(input_image, n=4, visualize=True) # Actually does the visualization

# Publishing as web-service 
### Webservice Deployment


<b>Prerequisites:</b> 
Please the check the **Prerequisites** section of our deployment notebook to set up your deployment CLI. You only need to set it up once for all your deployments. More deployment related topics including IoT Edge deployment can be found in the deployment notebook.
       
<b>Deployment API:</b>

> **Examples:**
- ```deploy_obj = AMLDeployment(deployment_name=deployment_name, associated_DNNModel=dnn_model, aml_env="cluster")``` # create deployment object
- ```deploy_obj.deploy()``` # deploy web service
- ```deploy_obj.status()``` # get status of deployment
- ```deploy_obj.score_image(local_image_path_or_image_url)``` # score an image
- ```deploy_obj.delete()``` # delete the web service
- ```deploy_obj.build_docker_image()``` # build docker image without creating webservice
- ```AMLDeployment.list_deployment()``` # list existing deployment
- ```AMLDeployment.delete_if_service_exist(deployment_name)``` # delete if the service exists with the deployment name

<b>Deployment management with portal:</b>

You can go to [Azure portal](https://ms.portal.azure.com/) to track and manage your deployments. From Azure portal, find your Machine Learning Model Management account page (You can search for your model management account name). Then go to: the model management account page->Model Management->Services.

Create the deployment by specifying an AMLDeployment object. Here the ranker is passed to the deployment object. 

In [ ]:
# ##### OPTIONAL - Interactive CLI setup helper ###### 
# # Interactive CLI setup helper, including model management account and deployment environment.
# # If you haven't setup you CLI before or if you want to change you CLI settings, you can use this block to help you interactively.
# # UNCOMMENT THE FOLLOWING LINES IF YOU HAVE NOT CREATED OR SET THE MODEL MANAGEMENT ACCOUNT AND DEPLOYMENT ENVIRONMENT

# from azuremltkbase.deployment import CliSetup
# CliSetup().run()

In [ ]:
# # Optional. Persist you model on disk and reuse it later for deployment. 
# from cvtk import CNTKTLModel, Context
# from cvtk.core.ranker import ImageSimilarityMetricRanker, ImageSimilarityLearnerRanker, ImageSimilarityRandomRanker, RankerEvaluation
# import os
# save_model_path = os.path.join(Context.get_global_context().storage.persistent_path, "saved_ranker.model")
# # Save model to disk
# ranker.save(save_model_path)
# # Load model from disk
# ranker = ImageSimilarityLearnerRanker.load(save_model_path)

In [ ]:
from cvtk.operationalization import AMLDeployment

# set deployment name
deployment_name = "imagesimilarity"

# Set a reference dataset
ranker.set_reference_data(test_set)

# Create deployment object
# It will use the current deployment environment (you can check it with CLI command "az ml env show").
deploy_obj = AMLDeployment(deployment_name=deployment_name, aml_env="cluster", associated_DNNModel=ranker, replicas=1)

# Alternatively, you can provide azure machine learning deployment cluster name (environment name) and resource group name
# to deploy your model. It will use the provided cluster to deploy. To do that, please uncomment the following lines to create 
# the deployment object.

# azureml_rscgroup = "<resource group>"
# cluster_name = "<cluster name>"
# deploy_obj = AMLDeployment(deployment_name=deployment_name, associated_DNNModel=ranker,
#                            aml_env="cluster", cluster_name=cluster_name, resource_group=azureml_rscgroup, replicas=1)

# Check if the webservice exists, if yes remove it first.
if deploy_obj.is_existing_service():
    AMLDeployment.delete_if_service_exist(deployment_name)
    
# create the webservice
print("Deploying to Azure cluster...")
deploy_obj.deploy()
print("Deployment DONE")

### Webservice comsumption

Once you created the webservice, you can score images with the deployed webservice. You have several options:

   - You can directly score the webservice with the deployment object with: deploy_obj.score_image(image_path_or_url) 
   - Or, you can use the Service endpoin url and Serivce key (None for local deployment) with: AMLDeployment.score_existing_service_with_image(image_path_or_url, service_endpoint_url, service_key=None)
   - Form your http requests directly to score the webservice endpoint (For advanced users).

#### Score with existing deployment object
```
deploy_obj.score_image(image_path_or_url)
```

In [ ]:
# Score with existing deployment object

# Score local image with file path
print("Score local image with file path")
image_path_or_url = test_set.images[0].storage_path
print("Image source:",image_path_or_url)
serialized_result_in_json = deploy_obj.score_image(image_path_or_url, image_resize_dims=[224,224])
print("serialized_result_in_json:", serialized_result_in_json)
# If you want to view the similar images
#re.visualize_similar_images_from_json(serialized_result_in_json)

# Score image url and remove image resizing
print("Score image url")
image_path_or_url = "https://cvtkdata.blob.core.windows.net/publicimages/microsoft_logo.jpg"
print("Image source:",image_path_or_url)
serialized_result_in_json = deploy_obj.score_image(image_path_or_url)
print("serialized_result_in_json:", serialized_result_in_json)


In [ ]:
# Time image scoring
import timeit

num_images = 10
for img_index, img_obj in enumerate(test_set.images[:num_images]):
    print("Calling API for image {} of {}: {}...".format(img_index, num_images, img_obj.name))
    tic = timeit.default_timer()
    return_json = deploy_obj.score_image(img_obj.storage_path, image_resize_dims=[224,224])
    print("   Time for API call: {:.2f} seconds".format(timeit.default_timer() - tic))
    print(return_json)

#### Score with service endpoint url and service key
```
    AMLDeployment.score_existing_service_with_image(image_path_or_url, service_endpoint_url, service_key=None)
```

In [ ]:
# Import related classes and functions
from cvtk.operationalization import AMLDeployment

service_endpoint_url = "" # please replace with your own service url
service_key = "" # please replace with your own service key
# score local image with file path
image_path_or_url = test_set.images[0].storage_path
print("Image source:",image_path_or_url)
serialized_result_in_json = AMLDeployment.score_existing_service_with_image(image_path_or_url,service_endpoint_url, service_key = service_key)
print("serialized_result_in_json:", serialized_result_in_json)

# score image url
image_path_or_url = "https://cvtkdata.blob.core.windows.net/publicimages/microsoft_logo.jpg"
print("Image source:",image_path_or_url)
serialized_result_in_json = AMLDeployment.score_existing_service_with_image(image_path_or_url,service_endpoint_url, service_key = service_key, image_resize_dims=[224,224])
print("serialized_result_in_json:", serialized_result_in_json)

#### Score endpoint with http request directly
Following is some example code to form the http request directly in Python. You can do it in other programming languages.

In [ ]:
def score_image_with_http(image, service_endpoint_url, service_key=None, parameters={}):
    """Score local image with http request

    Args:
        image (str): Image file path
        service_endpoint_url(str): web service endpoint url
        service_key(str): Service key. None for local deployment.
        parameters (dict): Additional request paramters in dictionary. Default is {}.


    Returns:
        str: serialized result 
    """
    import requests
    from io import BytesIO
    import base64
    import json

    if service_key is None:
        headers = {'Content-Type': 'application/json'}
    else:
        headers = {'Content-Type': 'application/json',
                   "Authorization": ('Bearer ' + service_key)}
    payload = []
    encoded = None
    
    # Read image
    with open(image,'rb') as f:
        image_buffer = BytesIO(f.read()) ## Getting an image file represented as a BytesIO object
        
    # Convert your image to base64 string
    # image_in_base64 : "b'{base64}'"
    encoded = base64.b64encode(image_buffer.getvalue())
    image_request = {"image_in_base64": "{0}".format(encoded), "parameters": parameters}
    payload.append(image_request)
    body = json.dumps(payload)
    r = requests.post(service_endpoint_url, data=body, headers=headers)
    try:
        result = json.loads(r.text)
        json.loads(result[0])
    except:
        raise ValueError("Incorrect output format. Result cant not be parsed: " + r.text)
    return result[0]

# Test with images
image = test_set.images[0].storage_path # A local image file
score_image_with_http(image, service_endpoint_url, service_key) # Local scoring the service_key is None

### Parse serialized result from webservice
The result from the webserice is in json string. You can parse it the with different DNN model classes

In [ ]:
image_path_or_url = "https://cvtkdata.blob.core.windows.net/publicimages/microsoft_logo.jpg"
print("Image source:",image_path_or_url)
serialized_result_in_json = deploy_obj.score_image(image_path_or_url, image_resize_dims=[224,224])
print("serialized_result_in_json:", serialized_result_in_json)

In [ ]:
# Parse result from json string
parsed_result = ImageSimilarityLearnerRanker.parse_serialized_result(serialized_result_in_json)
print("Parsed result:", parsed_result)

### Delete the webservice

In [ ]:
deploy_obj.delete()

© 2018 Microsoft. All rights reserved. 